In [37]:
import torch
from torch import nn
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from PIL import Image

from utils import render_rays, get_rays
from model import nerf, NGP, Plenoxels

import tqdm

In [24]:
load_model = torch.load('./pretrained/model_lego_800.pth',map_location=torch.device('cpu') )

/var/folders/4d/651vb1t15t79s6jcysz6pv140000gn/T/ipykernel_44742/4104461660.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_model = torch.load('./pretrained/model_l

In [26]:
nb_epochs = load_model['nb_epochs']
L = load_model['L']
H = load_model['H']
W = load_model['W']
F = load_model['F']
T = load_model['T']
N_min = load_model['N_min']
N_max = load_model['N_max']
num_samples = load_model['num_samples']

In [27]:
b = np.exp((np.log(N_max) - np.log(N_min)) / (L - 1))
Nl = [int(np.floor(N_min * b**l)) for l in range(L)]

In [28]:
model = NGP(T=T, Nl=Nl, device='cpu', scale=4)
model.load_state_dict(load_model['model_state_dict'])

<All keys matched successfully>

In [29]:
def render_rays(model, rays_d, rays_o, near, far, N):
    device = rays_d.device

    # Compute sampling points
    ts = torch.linspace(near, far, N,device=device).expand(rays_o.shape[0], N)
    mid = (ts[:, :-1] + ts[:, 1:]) / 2.
    lower = torch.cat((ts[:, :1], mid), -1)
    upper = torch.cat((mid, ts[:, -1:]), -1)
    u = torch.rand(ts.shape, device=device)
    ts = lower + (upper - lower) * u
    delta = torch.cat((ts[:, 1:] - ts[:, :-1], torch.tensor([1e10], device=device).expand(rays_o.shape[0], 1)), -1)
    
    # Prepare input for the model
    inputs = rays_o.unsqueeze(1) + rays_d.unsqueeze(1) * ts.unsqueeze(2)
    direcation = rays_d.expand(N, rays_d.shape[0], 3).transpose(0, 1).reshape(-1,3)
    inputs_flat = inputs.reshape(-1, 3)
    #print(.shape)

    # Process in batches
    colors, sigma = model(inputs_flat,direcation)
    colors = colors.reshape(inputs.shape)
    sigma = sigma.reshape(inputs.shape[:-1])

    # Volume rendering
    alpha = 1 - torch.exp(-sigma*delta)
    llf = torch.cumprod(1.0 - alpha + 1e-10, dim=-1)
    llf = torch.cat([torch.ones_like(llf[..., :1],device=device), llf[..., :-1]], dim=-1)
    weights = llf.unsqueeze(2) * alpha.unsqueeze(2)
    c = (weights * colors).sum(dim=1)
    weight_sum = weights.sum(-1).sum(-1)  # Regularization for white background 
    return c + 1 - weight_sum.unsqueeze(-1)


In [30]:
c2w =[]
for th in np.linspace(0., 360., 120, endpoint=False):
    theta = th
    phi = -60.
    trans_t = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 4.],
    [0, 0, 0, 1],
    ], dtype=np.float32)

    rot_phi = np.array([
    [1, 0, 0, 0],
    [0, np.cos(phi/180.*np.pi), -np.sin(phi/180.*np.pi), 0],
    [0, np.sin(phi/180.*np.pi),  np.cos(phi/180.*np.pi), 0],
    [0, 0, 0, 1],
    ], dtype=np.float32)

    rot_theta = np.array([
    [np.cos(theta/180.*np.pi), 0, -np.sin(theta/180.*np.pi), 0],
    [0, 1, 0, 0],
    [np.sin(theta/180.*np.pi), 0,  np.cos(theta/180.*np.pi), 0],
    [0, 0, 0, 1],
    ], dtype=np.float32)
    c2w.append(np.array([[-1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]], dtype=np.float32) @ rot_theta @ rot_phi @ trans_t)
    

In [44]:

for j, pose in enumerate(c2w[3:]):
    print(j+3)
    rays_d, rays_o = get_rays(H, W, 400, pose)
    chunk_size = 20
    device = 'cpu'
    px_values = []
    with torch.no_grad():
        for i in tqdm.tqdm(range(int(np.ceil(H / chunk_size)))):   # iterate over chunks
            ray_origins_ = rays_o[i * W * chunk_size: (i + 1) * W * chunk_size].to(device)
            ray_directions_ = rays_d[i * W * chunk_size: (i + 1) * W * chunk_size].to(device)
            px_values.append(render_rays(model, ray_directions_, ray_origins_,
                                            near=2., far=6., N=num_samples))
        img = torch.cat(px_values).data.cpu().numpy().reshape(H, W, 3)
        img = (img.clip(0, 1)*255).astype(np.uint8)
        img = Image.fromarray(img)
        img.save(f'output/lego/img_{j+3}.png')
            

3


100%|██████████| 40/40 [01:00<00:00,  1.51s/it]


4


100%|██████████| 40/40 [01:00<00:00,  1.51s/it]


5


...

100%|██████████| 40/40 [01:01<00:00,  1.53s/it]


118


100%|██████████| 40/40 [01:00<00:00,  1.52s/it]


119


100%|██████████| 40/40 [01:02<00:00,  1.55s/it]


In [45]:
!pip install "imageio[ffmpeg]"

In [46]:
import imageio as imageio
import os


image_folder = "./output/lego"
output_video = "./NGPlego.mp4"



fps = 30

with imageio.get_writer(output_video, fps=fps) as writer:
    for i in range(120):
        img_path = os.path.join(image_folder, f'img_{i}.png')
        img = imageio.imread(img_path)
        writer.append_data(img)

/var/folders/4d/651vb1t15t79s6jcysz6pv140000gn/T/ipykernel_44742/2955841100.py:15: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(img_path)


In [47]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('NGPlego.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls autoplay loop>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)